In [1]:
pip install s3fs

  Using cached botocore-1.19.52-py2.py3-none-any.whl (7.2 MB)
ERROR: boto3 1.17.34 has requirement botocore<1.21.0,>=1.20.34, but you'll have botocore 1.19.52 which is incompatible.
ERROR: awscli 1.19.34 has requirement botocore==1.20.34, but you'll have botocore 1.19.52 which is incompatible.
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.34
    Uninstalling botocore-1.20.34:
      Successfully uninstalled botocore-1.20.34
Note: you may need to restart the kernel to use updated packages.


In [56]:
%%writefile main_aggregazioni.py

import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import statsmodels.api as sm
# from scipy import stats
from datetime import datetime
import calendar as c

def lettura_temp_norm(file_t_norm):
#     df = pd.read_csv(file_t_norm)#,encoding='utf-16')
    df = costruzione_file_da_mensile(file_t_norm,inizio,fine)
#     df.rename(columns={'TIME - Date':'date','Codice Provincia':'provincia','Temperature Min':'temp_min','Temperature Max':'temp_max'},inplace=True)
    df = df[['DATA','PROVINCIA','TEMPERATURA_MAX','TEMPERATURA_MIN']]
    df.rename(columns={'DATA':'date','PROVINCIA':'provincia','TEMPERATURA_MIN':'temp_min','TEMPERATURA_MAX':'temp_max'},inplace=True)
    df = df[~df['provincia'].isnull()]
    df['date'] = pd.to_datetime(df['date'],format='%d/%m/%Y')
    df['daymonth'] = (df.date.dt.day).astype('str').str.pad(2, side='left', fillchar='0') + (df.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')
    df['T_mean'] = df[['temp_min', 'temp_max']].mean(axis=1)
    df.drop('Desc_Provincia',axis=1,inplace=True)
#     df.info()

    df = df.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, #  + pd.DateOffset(days=92)
    )
#     print(df)
    df['dayofyear'] = df.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    df['monthyear'] = (df.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df.year).astype('str')
    return df

def lettura_temp_norm_curve_min_max(nome_file):
    df = pd.read_csv(nome_file)#,encoding='utf-16')
#     print(df)
#     df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

#GIUSTO
    df['monthday'] = (df.mese).astype('str').str.pad(2, side='left', fillchar='0') +'-'+ (df.giorno).astype('str').str.pad(2, side='left', fillchar='0')
    df['date'] = "2020-" + df['monthday']
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

#DA TOGLIERE
#     df['mese'] = df.date.dt.month
#     df['giorno'] = df.date.dt.day
    
    col = [x for x in df.columns if 'T_' in x][0]
    if 'provincia' in df.columns:
        codice = 'provincia'
    else:
        codice = 'codice_oss'
    df = df.assign(
      dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
      year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    df['dayofyear'] = df.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    df.drop('date',axis=1,inplace=True)

    return df, col, codice

def lettura_osservatorio(file_oss):
    df_oss = pd.read_csv(file_oss)
    df_oss.rename(columns={'Cd Oss':'codice_oss','T Oss':'T_oss','Data':'date'},inplace=True)
#     df_oss = df_oss[df_oss['codice_oss']==11]
    df_oss['T_oss'] = df_oss.apply(lambda x: 18 if x['T_oss']==0 else 18-x['T_oss'],axis=1)
    df_oss['date'] = df_oss['date'].astype('str')
    df_oss['date'] = pd.to_datetime(df_oss['date'],format='%d/%m/%Y', exact=False)
    df_oss['daymonth'] = (df_oss.date.dt.day).astype('str').str.pad(2, side='left', fillchar='0') + (df_oss.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')
    df_oss = df_oss.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    df_oss['dayofyear'] = df_oss.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    df_oss.drop('Oss',axis=1,inplace=True)
    df_oss['monthyear'] = (df_oss.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df_oss.year).astype('str')
    return df_oss

def compute_yhat_prov(file_t_norm):

#     Calcolo temperature normali giornaliere    
    df = lettura_temp_norm(file_t_norm)
    df.rename(columns={'temp_max':'T_max_mean','temp_min':'T_min_mean','T_mean':'T_mean_m'},inplace=True)
    
    temp_norm = df.groupby(['dayofyear', 'provincia'])[['T_min_mean', 'T_max_mean','T_mean_m']].mean().unstack().reindex(range(367)).interpolate(method='linear').stack() #,'T_mean'
    province = temp_norm.reset_index(['provincia']).provincia.unique()
    finale = pd.DataFrame()

#     Fit armoniche di sesto grado per curve T_min, T_max e T_mean
    for p in province:
        temp_p = temp_norm.query('provincia == {}'.format(p)).reset_index(['provincia'])#.assign(std_mean=lambda x: x['std'], iv_l=lambda x: (x['mean'] - 2*x['std'].rolling(window=15, center=True).mean()), iv_h=lambda x: (x['mean'] + 2*x['std'].rolling(window=15, center=True).mean()))

        y_min = temp_p['T_min_mean'].values.ravel()
        y_max = temp_p['T_max_mean'].values.ravel()
        y_mean = temp_p['T_mean_m'].values.ravel()

        x = temp_p.index
        
        yhat_min = fit_harmonics(y_min, 6)
        yhat_max = fit_harmonics(y_max, 6)
        yhat_mean = fit_harmonics(y_mean, 6)

        temp_p['yhat_min'] = yhat_min
        temp_p['yhat_max'] = yhat_max
        temp_p['yhat_mean'] = yhat_mean

        finale = finale.append(temp_p)

    return finale.reset_index()

def fit_harmonics(y, n_harmonics):
    n = y.shape[0]
    X = np.matrix([*[[np.sin(i*k/n*2*np.pi) for i in range(n)] for k in range(n_harmonics)], *[[np.cos(i*k/n*2*np.pi) for i in range(n)] for k in range(n_harmonics)]]).T
    X = sm.add_constant(X)

    ols_model = sm.OLS(y, X).fit()
    yhat = ols_model.predict(X)
    
    return yhat

def aggregazione_serie_storiche_curve_min_max(nome_file,df_pesi,col_pesi_merge):
#     DA RICONTROLLARE COL FILE GIUSTO.
    
    df, col, codice = lettura_temp_norm_curve_min_max(nome_file)
    #df.drop('date',axis=1,inplace=True)
    print("Lettura file di temperature "+col+" per "+codice+".")
    
    print("Numero di "+codice+" presenti: "+str(df[codice].nunique()))
    
#     if col_pesi_merge==['mese','giorno']:
#         df_pesi.drop('date',axis=1,inplace=True)
    
    print("Procedura di ripartizione per peso avviata.")
    df_pesi = df.merge(df_pesi,on=col_pesi_merge+[codice],how='left')
    df_pesi['sum_pesi_gg'] = df_pesi.groupby(col_pesi_merge)['peso'].transform('sum')
    df_pesi[col+'_pesato'] = df_pesi[col]*df_pesi['peso']

    df_pesi[col + '_pesato'] = df_pesi.groupby(col_pesi_merge)[col + '_pesato'].transform('sum')

    df_pesi[col+'_pesato'] = df_pesi[col+'_pesato']/df_pesi['sum_pesi_gg']
    df_pesi = df_pesi[['date'] + col_pesi_merge+[col+'_pesato','sum_pesi_gg']]
    return df_pesi, col

def aggregazione_serie_storiche_norm_yhat(df,df_pesi,col_pesi_merge):
#     DA RICONTROLLARE COL FILE GIUSTO.
    print("Lettura file curva normale giornaliera da aggregare per provincia.")
    df = pd.read_csv(df)
    
    print("Numero province presenti: ",df.provincia.nunique())

    df = df[['provincia','yhat_mean']+col_pesi_merge]
    # df è il file di temperature max e min
    df_pesi = df.merge(df_pesi,on=col_pesi_merge+['provincia'],how='left')

    df_pesi['yhat_pesato'] = df_pesi['yhat_mean']*df_pesi['peso']
    
    df_pesi_mean_gg = df_pesi[['date','yhat_pesato']].groupby('date').sum().reset_index()

    df_pesi_mean_gg['mese'] = (df_pesi_mean_gg.date.dt.month).astype('int')
    df_pesi_mean_gg['giorno'] = (df_pesi_mean_gg.date.dt.day).astype('int')

    df_pesi_mean = df_pesi_mean_gg.copy()#[['daymonth',col+'_pesato']].groupby('daymonth').mean().reset_index()
    df_pesi_mean = df_pesi_mean.merge(df_pesi[col_pesi_merge+['sum_pesi_gg']],on=col_pesi_merge,how='left')
    df_pesi_mean['yhat_pesato'] = df_pesi_mean['yhat_pesato']/df_pesi_mean['sum_pesi_gg']
    df_pesi_mean = df_pesi_mean[['date','yhat_pesato','sum_pesi_gg']]
    df_pesi_mean.drop_duplicates(subset=['date'],inplace=True)
    return df_pesi_mean


def aggregazione_serie_storiche_oss_yhat(df,df_pesi,col_pesi_merge):
#     DA RICONTROLLARE COL FILE GIUSTO.
    print("Lettura file curva normale giornaliera da aggregare per osservatorio.")
    df = pd.read_csv(df)
    print("Numero osservatori presenti: ",df.codice_oss.nunique())

#     df['date_prov'] = '2020-'+df['mese'].astype('str').str.pad(2,'left','0')+'-'+df['giorno'].astype('str').str.pad(2,'left','0')
#     df.drop(['mese','giorno'],axis=1,inplace=True)
#     df['date_prov'] = pd.to_datetime(df['date_prov'],format='%Y-%m-%d')
#     df = df.assign(
#       dayofyear=lambda x: (x.date_prov).dt.dayofyear,
#     )

    df = df[['codice_oss','yhat']+col_pesi_merge]
    to_merge = df[col_pesi_merge].copy()

    df_pesi = df.merge(df_pesi,on=col_pesi_merge+['codice_oss'],how='left')
    df_pesi['yhat_pesato'] = df_pesi['yhat']*df_pesi['peso']
    df_pesi_mean_gg = df_pesi[col_pesi_merge+['yhat_pesato','date']].groupby(col_pesi_merge).sum().reset_index()
    df_pesi_mean_gg = df_pesi_mean_gg.merge(to_merge,on=col_pesi_merge,how='left')
    #df_pesi_mean_gg['monthday'] = (df_pesi_mean_gg['date_prov'].dt.month).astype('str').str.pad(2, side='left', fillchar='0')+'-'+(df_pesi_mean_gg['date_prov'].dt.day).astype('str').str.pad(2, side='left', fillchar='0')

    df_pesi_mean = df_pesi_mean_gg.copy()#[['daymonth',col+'_pesato']].groupby('daymonth').mean().reset_index()
    df_pesi_mean = df_pesi_mean.merge(df_pesi[col_pesi_merge+['sum_pesi_gg','date']],on=col_pesi_merge,how='left')
    df_pesi_mean['yhat_pesato'] = df_pesi_mean['yhat_pesato']/df_pesi_mean['sum_pesi_gg']
    df_pesi_mean = df_pesi_mean[['date','mese','giorno','yhat_pesato','sum_pesi_gg']]
    df_pesi_mean.drop_duplicates(subset=['mese','giorno'],inplace=True)
#     df.drop('date_prov',axis=1,inplace=True)
    
    return df_pesi_mean
    

def main_aggregazioni(file_input,pesi,path_to_output):
    print('Inizio funzione aggregazione')
    print('\n')
    
    my_bucket = 'zus-prod-s3'
    
    #idrun = '_'.join(file_input.split('/')[-2].split('_')[0:2]) + '_' + str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')
    
    df_pesi = pd.read_csv('s3://'+ my_bucket +'/'+pesi)
    if 'consumo_termico' in df_pesi.columns:
        df_pesi.rename(columns={'consumo_termico':'peso','osservatorio':'codice_oss'},inplace=True)
        prv = pd.DataFrame()
        prv['dayofyear'] = range(1,367)
        prv['date'] = pd.date_range('2020-01-01','2020-12-31').astype('str')#.str.slice(start=5)
        prv['date'] = pd.to_datetime(prv['date'],format='%Y-%m-%d')
        df_pesi = df_pesi.merge(prv,on='dayofyear',how='left')
    else:
        df_pesi['date'] = pd.to_datetime(df_pesi['date'],format='%d/%m/%Y') #'%Y-%m-%d') '%d/%m/%Y'

    df_pesi['mese'] = (df_pesi.date.dt.month).astype('int')
    df_pesi['giorno'] = (df_pesi.date.dt.day).astype('int')
    
    inizio_pesi = min(df_pesi['date']).strftime('%Y-%m-%d')
    fine_pesi = max(df_pesi['date']).strftime('%Y-%m-%d')
    idrun = inizio_pesi.replace('-','') + '_' + fine_pesi.replace('-','') + '_' + str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')

    df = pd.read_csv('s3://'+ my_bucket +'/'+file_input)
    list_nome_file = file_input.split('/')

    if 'provincia' in df.columns:
        
        col_pesi_merge = ['mese','giorno']

        if 'yhat_mean' in df.columns:
            df_pesi['sum_pesi_gg'] = df_pesi.groupby(['date'])['peso'].transform('sum')
            
            print('Aggregazione serie storiche temperature normali fittate (yhat_norm * pesi)')
            
            agg_ita = aggregazione_serie_storiche_norm_yhat('s3://'+ my_bucket +'/'+file_input,df_pesi,col_pesi_merge)
            file_csv = file_input.split('/')[-1]
            if 'max' in file_csv: tipo_yhat='max'
            elif 'min' in file_csv: tipo_yhat='min'
            else: tipo_yhat='mean'
            nome_output = 'agg_ita_norm_prov/'+idrun+'/agg_ita_norm_'+tipo_yhat+'_prov.csv'
            
            metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_NORM_PROV']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+'agg_ita_norm_prov/best/agg_ita_norm_'+tipo_yhat+'_prov.csv']})
            metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_norm_prov/'+idrun+'/metadati.csv',index=False)

        else:
            print('Aggregazione curve min_max periodo (curve * pesi)')
            
            agg_ita, col = aggregazione_serie_storiche_curve_min_max('s3://'+ my_bucket +'/'+file_input,df_pesi,col_pesi_merge)
            agg_ita.drop_duplicates(subset=['mese','giorno',col + '_pesato'],inplace=True) 
#             name = file_input.split('/')[-1]
#             name = file_input.replace('.csv','')
#             name = name.split('_')[-4:]
#             name = '_'.join(name)
            
            if 'T_min' in df.columns:
                nome_output = 'agg_ita_min_periodo_prov/'+idrun+'/agg_ita_min_periodo_prov.csv'
                metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_MIN_PERIODO_PROV']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+nome_output]})
                metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_min_periodo_prov/'+idrun+'/metadati.csv',index=False)
            else:
                nome_output = 'agg_ita_max_periodo_prov/'+idrun+'/agg_ita_max_periodo_prov.csv'
                metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_MAX_PERIODO_PROV']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+nome_output]})
                metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_max_periodo_prov/'+idrun+'/metadati.csv',index=False)

#         agg_ita['mese'] = agg_ita['date'].dt.month
#         agg_ita['giorno'] = agg_ita['date'].dt.day
#         print("prov: ",agg_ita)

        agg_ita.to_csv('s3://'+ my_bucket +'/'+path_to_output+nome_output,index=False)
    
#         pesi_mean_norm.to_csv(path_output+'agg_ita/best/'+nome_output+'_T_mean_p.csv',index=False)
#         pesi_yhat_norm.to_csv(path_output+'agg_ita/best/'+nome_output+'_yhat_p.csv',index=False)
        #agg_ita.to_csv('prova_agg_prov.csv',index=False)
    
    else:
        
        col_pesi_merge = ['mese','giorno']#['dayofyear']
        
        if 'yhat' in df.columns:
            df_pesi['sum_pesi_gg'] = df_pesi.groupby(['dayofyear'])['peso'].transform('sum')
            df_pesi.drop('dayofyear',axis=1,inplace=True)

            print('Aggregazione serie storiche temperature normali fittate (yhat_norm * pesi)')
            agg_ita = aggregazione_serie_storiche_oss_yhat('s3://'+ my_bucket +'/'+file_input,df_pesi,col_pesi_merge)
            file_csv = file_input.split('/')[-1]
            if 'max' in file_csv: tipo_yhat='max'
            elif 'min' in file_csv: tipo_yhat='min'
            else: tipo_yhat='mean'
            nome_output = 'agg_ita_norm_oss/'+idrun+'/agg_ita_norm_'+tipo_yhat+'_oss.csv'
            
            metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_NORM_OSS']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+'agg_ita_norm_oss/best/agg_ita_norm_'+tipo_yhat+'_oss.csv']})

            metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_norm_oss/'+idrun+'/metadati.csv',index=False)
           
        else:
            print('Aggregazione curve min_max periodo (curve * pesi)')
            df_pesi.drop('dayofyear',axis=1,inplace=True)

            agg_ita, col = aggregazione_serie_storiche_curve_min_max('s3://'+ my_bucket +'/'+file_input,df_pesi,col_pesi_merge)
            agg_ita.drop_duplicates(subset=['mese','giorno',col + '_pesato'],inplace=True)
#             name = file_input.split('/')[-1]
#             print(name)
#             name = file_input.replace('.csv','')
#             name = name.split('_')[-4:]
#             name = '_'.join(name)
            if 'min' in list_nome_file[-1]:
                nome_output = 'agg_ita_min_periodo_oss/'+idrun+'/agg_ita_min_periodo_oss.csv'
                metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_MIN_PERIODO_PROV']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+nome_output]})
                metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_min_periodo_oss/'+idrun+'/metadati.csv',index=False)
            else:
                nome_output = 'agg_ita_max_periodo_oss/'+idrun+'/agg_ita_max_periodo_oss.csv'
                metadatati = pd.DataFrame(data={'MODELLO':['AGG_ITA_MIN_PERIODO_PROV']*3,'ID_RUN':['s3://'+ my_bucket +'/'+path_to_output+nome_output]*3,'NOME_PARAMETRO':['FILE_INPUT','PESI','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_input,pesi,path_to_output+nome_output]})
                metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/agg_ita_max_periodo_oss/'+idrun+'/metadati.csv',index=False)
        
#         agg_ita['mese'] = agg_ita['monthday'].str.slice(stop=2)
#         agg_ita['giorno'] = agg_ita['monthday'].str.slice(start=3)
#         agg_ita.drop(['monthday'],axis=1,inplace=True)
#         print("oss: ",agg_ita)
#         agg_ita = agg_ita[['mese','giorno','yhat_pesato','sum_pesi_gg']]

        agg_ita.to_csv('s3://'+ my_bucket +'/'+path_to_output+nome_output,index=False)
#         agg_ita.to_csv('prova_pesi_oss_yhat.csv',index=False)
    print("Procedura terminata.")



Overwriting main_aggregazioni.py


In [54]:
main_aggregazioni('./max_periodo_prov/200011_2020320210430084751/max_periodo.csv','preprocessato/sistema/temperature/external/file_config/pesi_temperature_provincia.csv','./')
##### NUOVO PATH: 'preprocessato/sistema/temperatura/epson/temperatura'

# main_aggregazioni('./mean_d_oss/best/mean_d_oss_20000101_20201231.csv','preprocessato/sistema/temperature/external/file_config/pesi_osservatorio.csv','./')

Inizio funzione aggregazione


Aggregazione curve min_max periodo (curve * pesi)
Lettura file di temperature T_max per provincia.
Numero di provincia presenti: 103
Procedura di ripartizione per peso avviata.
Procedura terminata.


In [13]:
##agg yhat prov
df=pd.read_csv('s3://zus-prod-s3/./agg_ita_norm_prov/20201001_20210930_20210430153315/agg_ita_norm_mean_prov.csv')
df

,date,yhat_pesato,sum_pesi_gg
0,2020-10-01,16.372021,61012.56
1,2020-10-02,16.213706,59846.21
2,2020-10-03,16.083742,50884.64
3,2020-10-04,15.956167,74279.95
4,2020-10-05,15.763682,110692.15
...,...,...,...
360,2021-09-26,17.281919,36763.39
361,2021-09-27,17.082498,40653.14
362,2021-09-28,16.914582,40653.14
363,2021-09-29,16.746193,40653.14


In [41]:
##agg yhat oss
df = pd.read_csv('s3://zus-prod-s3/./agg_ita_norm_oss/20200101_20201231_20210430155000/agg_ita_norm_mean_oss.csv')
df

,date,mese,giorno,yhat_pesato,sum_pesi_gg
0,2020-01-01,1,1,4.893271,10870226.87
1,2020-01-02,1,2,4.916797,10463725.80
2,2020-01-03,1,3,4.924331,10292466.45
3,2020-01-04,1,4,4.915458,12201191.71
4,2020-01-05,1,5,4.890546,12584855.58
...,...,...,...,...,...
361,2020-12-27,12,27,4.934638,11210169.76
362,2020-12-28,12,28,4.886945,13549053.07
363,2020-12-29,12,29,4.803718,13273426.41
364,2020-12-30,12,30,4.783585,12769365.60


In [53]:
#max_periodo oss
df = pd.read_csv('s3://zus-prod-s3/./agg_ita_max_periodo_oss/20200101_20201231_20210430160010/agg_ita_max_periodo_oss.csv')
df

,date,mese,giorno,T_oss_pesato,sum_pesi_gg
0,2020-11-01,11,1,5.840830,4985529.89
1,2020-11-02,11,2,6.934365,5811533.94
2,2020-11-03,11,3,6.077228,5411733.53
3,2020-11-04,11,4,6.648215,4993502.20
4,2020-11-05,11,5,5.748887,5026077.96
...,...,...,...,...,...
146,2020-03-27,3,27,7.401382,6138134.18
147,2020-03-28,3,28,7.282950,5289071.79
148,2020-03-29,3,29,6.380479,5289071.79
149,2020-03-30,3,30,7.985894,5289071.79


In [55]:
##max periodo oss
df = pd.read_csv('s3://zus-prod-s3/./agg_ita_max_periodo_prov/20201001_20210930_20210430160206/agg_ita_max_periodo_prov.csv')
df

,date,mese,giorno,T_max_pesato,sum_pesi_gg
0,2020-11-01,11,1,18.569637,4728118.35
1,2020-11-02,11,2,15.688982,5528785.98
2,2020-11-03,11,3,16.444362,5153818.60
3,2020-11-04,11,4,15.115535,4769805.69
4,2020-11-05,11,5,15.778402,4799122.81
...,...,...,...,...,...
146,2021-03-27,3,27,14.676287,5910285.58
147,2021-03-28,3,28,16.534011,5854944.24
148,2021-03-29,3,29,17.619028,5058770.70
149,2021-03-30,3,30,16.738288,5058770.70


In [10]:
df_old = pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/mean_d_prov/best/mean_d_prov_20000101_20201231.csv')
df_old[df_old['monthday']=="01-01"]

,provincia,monthday,T_mean,T_min_abs,T_max_abs,gradi_giorno_yhat_mean,gradi_giorno_T_mean,std,var,yhat,percentile_10_T_mean,percentile_90_T_mean,percentile_10_yhat,percentile_90_yhat,c_i_95_T_mean_l,c_i_95_T_mean_u,c_i_95_yhat_l,c_i_95_yhat_u
9476,1,01-01,2.30,-5,10.0,15.466278,15.70,2.745843,7.539656,2.533722,-1.220171,5.820171,-0.986449,6.053893,1.860574,2.739426,2.094295,2.973148
9477,2,01-01,1.90,-7,8.0,15.980534,16.10,2.690717,7.239956,2.019466,-1.549499,5.349499,-1.430033,5.468964,1.469396,2.330604,1.588861,2.450070
9478,3,01-01,1.90,-7,8.0,15.980534,16.10,2.690717,7.239956,2.019466,-1.549499,5.349499,-1.430033,5.468964,1.469396,2.330604,1.588861,2.450070
9479,4,01-01,0.85,-9,8.0,16.800743,17.15,3.204552,10.269156,1.199257,-3.258236,4.958236,-2.908979,5.307493,0.337165,1.362835,0.686422,1.712092
9480,5,01-01,2.00,-6,9.0,15.714037,16.00,2.683149,7.199289,2.285963,-1.439797,5.439797,-1.153834,5.725760,1.570607,2.429393,1.856570,2.715356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9574,99,01-01,4.25,-5,18.0,13.356534,13.75,2.919758,8.524989,4.643466,0.506870,7.993130,0.900336,8.386596,3.782741,4.717259,4.176207,5.110724
9575,100,01-01,6.50,-1,15.0,11.213907,11.50,2.993593,8.961600,6.786093,2.662214,10.337786,2.948307,10.623879,6.020925,6.979075,6.307018,7.265168
9576,101,01-01,10.15,3,17.0,7.919250,7.85,2.552343,6.514456,10.080750,6.877896,13.422104,6.808646,13.352854,9.741540,10.558460,9.672290,10.489210
9577,102,01-01,8.70,-3,16.0,9.180202,9.30,2.481955,6.160100,8.819798,5.518134,11.881866,5.637932,12.001664,8.302804,9.097196,8.422602,9.216993


In [5]:
import pandas as pd
max_oss = pd.read_csv('s3://zus-prod-s3/./max_periodo_oss/200011_2020320210430092143/max_periodo.csv')
max_oss

,codice_oss,date,T_oss,to_groupby,mese,giorno,dayofyear,year
0,11,2009-11-01,8.0,2010,11,1,306,2009
1,11,2009-11-02,13.0,2010,11,2,307,2009
2,11,2009-11-03,8.5,2010,11,3,308,2009
3,11,2009-11-04,11.5,2010,11,4,309,2009
4,11,2009-11-05,10.5,2010,11,5,310,2009
...,...,...,...,...,...,...,...,...
2713,29,2012-03-27,6.0,2012,3,27,87,2012
2714,29,2012-03-28,6.5,2012,3,28,88,2012
2715,29,2012-03-29,4.0,2012,3,29,89,2012
2716,29,2012-03-30,4.0,2012,3,30,90,2012
